In [1]:
import pandas as pd
import numpy as np

import import_ipynb
import preprocessamentofuncoes as pp

importing Jupyter notebook from preprocessamentofuncoes.ipynb


### Definindo as Funções Auxiliares

In [2]:
# Excluindo os dias vazios
def removerAtributosFaltosos(covidData, atributo):
    instanciasVazias = covidData.loc[covidData[atributo].isnull()]
    covidData = covidData.drop(instanciasVazias.index)
    return covidData

In [3]:
def converterDiasSintomas(covidData, diasSintomas, novaColuna):
    covidData.loc[((covidData[diasSintomas] >= 0) & (covidData[diasSintomas] <= 3)), novaColuna] = 0
    covidData.loc[((covidData[diasSintomas] >= 4) & (covidData[diasSintomas] <= 6)), novaColuna] = 1
    covidData.loc[((covidData[diasSintomas] >= 7) & (covidData[diasSintomas] <= 9)), novaColuna] = 2
    covidData.loc[((covidData[diasSintomas] >= 10) & (covidData[diasSintomas] <= 12)), novaColuna] = 3
    covidData.loc[((covidData[diasSintomas] >= 13) & (covidData[diasSintomas] <= 15)), novaColuna] = 4
    covidData.loc[((covidData[diasSintomas] >= 16) & (covidData[diasSintomas] <= 18)), novaColuna] = 5
    covidData.loc[((covidData[diasSintomas] > 18)), novaColuna] = 6
    return covidData

In [4]:
def salvarDataframeEmPlanilha(covidDataBalanced, url):
    write = pd.ExcelWriter(url)
    covidDataBalanced.to_excel(write, 'dados', index = False)
    write.save()

In [32]:
def balanceandoDados(urlBase, colunasRemovidas, uf):
    covidData = pd.read_excel(urlBase)
    print('Base Total', covidData['evolucaoCaso'].value_counts())
    
    # Removendo as colunas destacadas
    covidData = covidData.drop(columns = colunasRemovidas)
    
    # Excluindo os dias vazios
    covidData = removerAtributosFaltosos(covidData, 'diasSintomas')
    
    # Removendo os dias outliers
    diasSintomasOutliers = covidData.loc[((covidData['diasSintomas'] > 30))]
    covidData = covidData.drop(diasSintomasOutliers.index)
    
    # Converter os dias de sintomas em faixas
    covidData = converterDiasSintomas(covidData, 'diasSintomas', 'faixaDiasSintomas')
    
    # Removendo o atributo Dias de Sintomas 
    covidData = covidData.drop(columns = 'diasSintomas')
    
    # Balanceamento - OC 60/40
    covidDataBalanced = pp.balanceamento(covidData, 'evolucaoCaso', 0, 40, 1)
    covidDataBalanced = removerAtributosFaltosos(covidDataBalanced, 'faixaDiasSintomas')
    salvarDataframeEmPlanilha(covidDataBalanced, 'PreProcessamento/balanceadas/OC/ba-oc-6040-' + uf + '.xlsx')
    print('OC 6040 - ', covidDataBalanced.shape)
    
    # Balanceamento - OC 70/30
    covidDataBalanced = pp.balanceamento(covidData, 'evolucaoCaso', 0, 30, 1)
    covidDataBalanced = removerAtributosFaltosos(covidDataBalanced, 'faixaDiasSintomas')
    salvarDataframeEmPlanilha(covidDataBalanced, 'PreProcessamento/balanceadas/OC/ba-oc-7030-' + uf + '.xlsx')
    print('OC 7030 - ', covidDataBalanced.shape)

    # Balanceamento - IC 60/40
    covidDataBalanced = pp.balanceamento(covidData, 'evolucaoCaso', 2, 40, 1)
    covidDataBalanced = removerAtributosFaltosos(covidDataBalanced, 'faixaDiasSintomas')
    salvarDataframeEmPlanilha(covidDataBalanced, 'PreProcessamento/balanceadas/IC/ba-ic-6040-' + uf + '.xlsx')
    print('IC 6040 - ', covidDataBalanced.shape)

    # Balanceamento - IC 70/30
    covidDataBalanced = pp.balanceamento(covidData, 'evolucaoCaso', 2, 30, 1)
    covidDataBalanced = removerAtributosFaltosos(covidDataBalanced, 'faixaDiasSintomas')
    salvarDataframeEmPlanilha(covidDataBalanced, 'PreProcessamento/balanceadas/IC/ba-ic-7030-' + uf + '.xlsx')
    print('IC 7030 - ', covidDataBalanced.shape)
  
    print("Finalizado - Base ", uf)

### Realizando o balanceamento

In [35]:
colunasRemovidas = ['source_id', 'dataInicioSintomas', 'dataNotificacao', 'idade',
                    'sintomas', 'outrosSintomas', 'condicoes', 'outrasCondicoes',
                    'dataPrimeiraDose', 'dataSegundaDose', 'classificacaoFinal', 
                    'outrosSintomas', 'outrasCondicoes']

In [51]:
bases = ['sus-ac-pp.xlsx', 'sus-al-pp.xlsx', 'sus-am-pp.xlsx', 'sus-ap-pp.xlsx', 
         'sus-ba-pp.xlsx', 'sus-ce-pp.xlsx', 'sus-df-pp.xlsx', 
         'sus-go-pp.xlsx', 'sus-ma-pp.xlsx', 'sus-mg-pp.xlsx', 'sus-ms-pp.xlsx', 
         'sus-mt-pp.xlsx', 'sus-pa-pp.xlsx', 'sus-pb-pp.xlsx', 'sus-pe-pp.xlsx', 
         'sus-pi-pp.xlsx',  'sus-rj-pp.xlsx', 'sus-rn-pp.xlsx', 
         'sus-ro-pp.xlsx', 'sus-rr-pp.xlsx', 'sus-rs-pp.xlsx', 'sus-sc-pp.xlsx', 
         'sus-se-pp.xlsx', 'sus-to-pp.xlsx']


ufs = ['ac', 'al', 'am', 'ap', 'ba', 'ce', 'df',  
      'go', 'ma', 'mg', 'ms', 'mt', 'pa', 'pb', 'pe',
      'pi', 'rj', 'rn', 'ro', 'rr', 'rs', 'sc', 
       'se', 'to']


urlPadrao = 'PreProcessamento/'

In [41]:
for i in range(0, len(bases)):
    url = urlPadrao + bases[i]
    balanceandoDados(url, colunasRemovidas, ufs[i])

Base Total 1    16530
0      188
2       10
Name: evolucaoCaso, dtype: int64
OC 6040 -  (574, 38)
OC 7030 -  (710, 38)
IC 6040 -  (28, 38)
IC 7030 -  (34, 38)
Finalizado - Base  ma
Base Total 1    369125
0      1322
2       234
Name: evolucaoCaso, dtype: int64
OC 6040 -  (4254, 38)
OC 7030 -  (5263, 38)
IC 6040 -  (703, 38)
IC 7030 -  (871, 38)
Finalizado - Base  mg
Base Total 1    142604
2        63
0        24
Name: evolucaoCaso, dtype: int64
OC 6040 -  (77, 38)
OC 7030 -  (95, 38)
IC 6040 -  (196, 38)
IC 7030 -  (242, 38)
Finalizado - Base  ms
Base Total 1    55119
0      210
2       28
Name: evolucaoCaso, dtype: int64
OC 6040 -  (682, 38)
OC 7030 -  (845, 38)
IC 6040 -  (91, 38)
IC 7030 -  (112, 38)
Finalizado - Base  mt
Base Total 1    66601
0      683
2       80
Name: evolucaoCaso, dtype: int64
OC 6040 -  (2124, 38)
OC 7030 -  (2629, 38)
IC 6040 -  (234, 38)
IC 7030 -  (290, 38)
Finalizado - Base  pa
Base Total 1    82601
2      162
0      157
Name: evolucaoCaso, dtype: int64
OC 

KeyboardInterrupt: 

In [33]:
# balanceandoDados(urlBase, colunasRemovidas, uf)


Base Total 1    23760
0       92
2       18
Name: evolucaoCaso, dtype: int64
OC 6040 -  (301, 38)
OC 7030 -  (372, 38)
IC 6040 -  (59, 38)
IC 7030 -  (73, 38)
Finalizado - Base  ac


In [52]:
dfMetricas = pd.DataFrame()
for i in range(0, len(bases)):
    print('base ', ufs[i])
    row = {}
    url = urlPadrao + bases[i]
    covidData = pd.read_excel(url)
    evolucao = covidData['evolucaoCaso'].value_counts()

    row['UF'] = ufs[i].upper()
    row['Instâncias'] = covidData.shape[0]
    row['Curado'] = evolucao[1]
    row['Interado'] = evolucao[2]
    row['Óbito'] = evolucao[0]

    dfMetricas = pd.concat([dfMetricas, pd.DataFrame([row])])

base  ac
base  al
base  am
base  ap
base  ba
base  ce
base  df
base  go
base  ma
base  mg
base  ms
base  mt
base  pa
base  pb
base  pe
base  pi
base  rj
base  rn
base  ro
base  rr
base  rs
base  sc
base  se
base  to


In [54]:
salvarDataframeEmPlanilha(dfMetricas, 'metricas.xlsx')